In [1]:
import pygame
import math
from queue import PriorityQueue

SIZE = 800 #Window is going to be a square so we only need one variable for both Length and Width
WIN = pygame.display.set_mode((SIZE, SIZE)) #Setting Window Size
pygame.display.set_caption("A* Pathfinding Algorithm Visualizer") #Window Title

#Defining RGB Color Codes for our Cells
#THe Color of the cell will let us know what kind of cell/what attributes a cell has
BLACK = (0 , 0, 0) #Barrier
WHITE = (255, 255, 255) #Empty
GRAY = (128, 128, 128) #Grid Lines
RED = (255, 0 , 0) #Closed
GREEN = (0, 255, 0) #Open
ORANGE = (255, 165, 0) #Start
PURPLE = (128, 0 , 128) #End
TURQUOISE = (64, 224, 208) #Path

class Cell:
    def __init__(self, row, col, size, total_rows):
        self.row = row
        self.col = col
        self.x = row * size
        self.y = col * size
        self.color = WHITE
        self.neighbors = []
        self.size = size
        self.total_rows = total_rows

    #Gets the position of the cell
    def get_pos(self):
        return self.row, self.col
    
    #Methods that check the type of cell
    def is_closed(self):
        return self.color == RED
    
    def is_open(self):
        return self.color == GREEN
    
    def is_barrier(self):
        return self.color == BLACK
    
    def is_start(self):
        return self.color == ORANGE

    def is_end(self):
        return self.color == PURPLE

    #Methods that set/make cells 
    def make_closed(self):
        self.color = RED
    
    def make_open(self):
        self.color = GREEN
    
    def make_barrier(self):
        self.color = BLACK
    
    def make_start(self):
        self.color = ORANGE

    def make_end(self):
        self.color = PURPLE
    
    def make_path(self):
        self.color = TURQUOISE

    #Resets cell back to empty/white
    def reset(self):
        self.color = WHITE

    #Draws Cell
    def draw(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y, self.size, self.size))

    #Checks and updates neighboring Cells
    def update_neighbors(self, grid):
        self.neighbors = []

        #DOWN
        if self.row < self.total_rows - 1 and not grid[self.row + 1][self.col].is_barrier():
            self.neighbors.append(grid[self.row + 1][self.col])

        #UP
        if self.row > 0 and not grid[self.row - 1][self.col].is_barrier():
            self.neighbors.append(grid[self.row - 1][self.col])
        
        #RIGHT
        if self.col < self.total_rows - 1 and not grid[self.row][self.col + 1].is_barrier():
            self.neighbors.append(grid[self.row][self.col + 1])

        #LEFT
        if self.row > 0 and not grid[self.row][self.col - 1].is_barrier():
            self.neighbors.append(grid[self.row][self.col - 1])

    def __lt__(self, other):
        return False

#Defining our heuristic to calculate the distance between two points (p1 and p2)
def h(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return abs(x1 - x2) + abs(y1 - y2)

#Reconstructs shortest path between start and end to draw
def reconstruct_path(came_from, current, draw):
    while current in came_from:
        current = came_from[current]
        current.make_path()
        draw()

#Algorithm Logic
def algorithm(draw, grid, start, end):
    count = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    came_from = {}

    g_score = {cell: float("inf") for row in grid for cell in row}
    g_score[start] = 0
    f_score = {cell: float("inf") for row in grid for cell in row}
    f_score[start] = h(start.get_pos(), end.get_pos())

    open_set_hash = {start}

    while not open_set.empty():
        #Allows user to quit program while algorithm is running
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        
        current = open_set.get()[2]
        open_set_hash.remove(current)

        #Makes path if current cell is the end cell
        if current == end:
            reconstruct_path(came_from, end, draw)
            end.make_end()
            return True

        for neighbor in current.neighbors:
            temp_g_score = g_score[current] + 1

            if temp_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = temp_g_score
                f_score[neighbor] = temp_g_score + h(neighbor.get_pos(), end.get_pos())
                if neighbor not in open_set_hash:
                    count += 1
                    open_set.put((f_score[neighbor], count, neighbor))
                    open_set_hash.add(neighbor)
                    neighbor.make_open()

        draw()

        if current != start:
            current.make_closed()

    return False        

#Makes grid of cells
def make_grid(rows, size):
    grid = []
    gap = size // rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            cell = Cell(i , j, gap, rows)
            grid[i].append(cell)

    return grid

#Draws the grid lines onto the Window
def draw_grid(win, rows, size):
    gap = size // rows
    #For loop draws vertical lines
    for i in range(rows):
        pygame.draw.line(win, GRAY, (0, i * gap), (size, i * gap))
        #Draws horizontal lines
        for j in range(rows):
            pygame.draw.line(win, GRAY, (j * gap, 0), (j * gap, size))

def draw(win, grid, rows, size):
    win.fill(WHITE)

    for row in grid:
        for cell in row:
            cell.draw(win)
    
    draw_grid(win, rows, size)
    pygame.display.update()

def get_clicked_pos(pos, rows, size):
    gap = size // rows
    y, x = pos

    row = y // gap
    col = x // gap
    return row, col

#MAIN LOOP
def main(win, size):
    ROWS = 50
    grid = make_grid(ROWS, size)

    #Start/End Cells
    start_cell = None
    end_cell = None

    run = True

    #Main loop
    while run:
        draw(win, grid, ROWS, size)
        #Checks for different types of events that may happen
        for event in pygame.event.get():
            #Quit Event
            if event.type == pygame.QUIT:
                run = False

            #Left Mouse Click
            if pygame.mouse.get_pressed()[0]:
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, size)
                cell = grid[row][col]

                #If Start cell does not exist, make it
                if not start_cell and cell != end_cell:
                    start_cell = cell
                    start_cell.make_start()

                #If End cell does not exist, make it
                elif not end_cell and cell != start_cell:
                    end_cell = cell
                    end_cell.make_end()

                #make barrier cells
                elif cell != end_cell and cell != start_cell:
                    cell.make_barrier()

            #Right Mouse Click/Erase
            elif pygame.mouse.get_pressed()[2]:
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, size)
                cell = grid[row][col]
                cell.reset()

                if cell == start_cell:
                    start_cell = None
                elif cell == end_cell:
                    end_cell = None

            #SPACEBAR starts the pathfinding algorithm
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and start_cell and end_cell:
                    for row in grid:
                        for cell in row:
                            cell.update_neighbors(grid)

                    algorithm(lambda: draw(win, grid, ROWS, size), grid, start_cell, end_cell)

    pygame.quit()
main(WIN, SIZE)

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import math
import heapq
import pygame
import time

# TODO
# Add Popup windows/menus
# add options to pre-genterate a maze for the user
# resize window to add more nodes

# tuples for RGB values of the screen and different node states. subject to change while searching for best clarity
# white for the default nodes
# Grey for the walls or obstacles
# black for node margins
# Orange for nodes in the open set
# green for start node
# red for end node
# blue for the closed set
# yellow for the best path
NODE_COLOUR = (255, 255, 255)
WALL_COLOUR = (47,79,79)
MARGIN_COLOUR = (0,0,0)
OPEN_COLOUR = (255,178,102)
START_COLOUR = (0, 255, 0)
CLOSED_COLOUR = (82, 191, 255)
END_COLOUR = (213, 50, 80)
BEST_PATH_COLOUR = (243, 255, 82)

WIDTH = 20
HEIGHT = 20
MARGIN = 2

SCREEN_SIZE = (883, 883)

# start, end and current nodes for referencing 
start = None
current = None
end = None

# open and closed set. Open is a min heap using heapq
# closed is regular list
open = []
closed = []
# initialize empty grid
grid = []
# boolean to check if the program has already been run,
#  if it has, will reset only certain nodes to be rerun
rerun = False
# boolean to check if the program has been ended
done = False

# node class
class Node:
    # initialize node attributes
    def __init__(self, row, col, width, height):
        self.row = row
        self.col = col
        self.x = row * width
        self.y = col * width
        self.width = width
        self.height = height
        self.colour = NODE_COLOUR
        self.parent = None
        self.gcost = math.inf
        self.hcost = 0
        self.fcost = 0
    # return position of self. grid[i][i].getMyPosition
    def getMyPosition(self):
        return self.row, self.col
    # node draws itself
    def draw(self, display):
        pygame.draw.rect(display, self.colour,
        [(MARGIN + WIDTH) * self.col + MARGIN,
         (MARGIN + HEIGHT) * self.row + MARGIN, WIDTH, HEIGHT])
        pygame.display.update()
    # used when Python compares one node against another in a heap
    def __lt__(self, other):
        # if self.fcost == other.fcost:
        #     return self.hcost > other.hcost
        return self.fcost < other.fcost
    # get neighbors of self on provided grid
    #TODO enable diagonal movement?
    def getNeighbors(self, grid):
        directions = [[1, 0], [0, 1], [-1, 0], [0, -1]]
        result = []
        for direction in directions:
            newRow = (self.row + direction[0])
            newCol = (self.col + direction[1])
            if newRow > -1 and newCol > -1 and newRow < 40 and newCol < 40:
                neighbor = grid[newRow][newCol]
                if neighbor.isWall() == False:
                    result.append(neighbor)
        # result = [item for item in result if item.col > -1 and item.row > -1]
        return result
    # set of methods to change Node colour based on node type
    def makeStart(self):
        self.colour = START_COLOUR
    def makeEnd(self):
        self.colour = END_COLOUR
    def makeWall(self):
        self.colour = WALL_COLOUR
    def makeOpen(self):
        self.colour = OPEN_COLOUR
    def makeClosed(self):
        self.colour = CLOSED_COLOUR
    def makeBest(self):
        self.colour = BEST_PATH_COLOUR
    # set of methods to chek if the node is in a particular state
    def isStart(self):
        return self.colour == START_COLOUR
    def isEnd(self):
        return self.colour == END_COLOUR
    def isWall(self):
        return self.colour == WALL_COLOUR
    def isOpen(self):
        return self.colour == OPEN_COLOUR
    def isClosed(self):
        return self.colour == CLOSED_COLOUR
    def isBest(self):
        return self.colour == BEST_PATH_COLOUR
    #reset node to default white
    def reset(self):
        self.colour = NODE_COLOUR

# get position of node at mouse position
def getPosition():
    position = pygame.mouse.get_pos()
    column = position[0] // (WIDTH + MARGIN)
    row = position[1] // (HEIGHT + MARGIN)
    coords = (row, column)
    return coords

# set of action to take on mouse LEFT click
# checks to see if node is start or end to allow replacing of them
# then turns clicked nodes into walls
def placeWalls(start, end, grid):
    row, col = getPosition()
    if grid[row][col].isStart():
        start = None
        grid[row][col].makeWall()
    elif grid[row][col].isEnd():
        end = None
        grid[row][col].makeWall()
    else:
        grid[row][col].makeWall()

# set of action to take on mouse RIGHT click
# checks to see if node is start or end to allow replacing of them
# then resets clicked node
def resetNode(start, end, grid):
    row, col = getPosition()
    if grid[row][col].isStart():
        start = None
        grid[row][col].reset()
    elif grid[row][col].isEnd():
        end = None
        grid[row][col].reset()
    else:
        grid[row][col].reset()

# draw nodes on screen and draw colors based on class method returns
def draw():
    for row in range(40):
        for column in range(40):
            colour = NODE_COLOUR
            if grid[row][column].isWall():
                colour = WALL_COLOUR
            if grid[row][column].isStart():
                colour = START_COLOUR
            if grid[row][column].isEnd():
                colour = END_COLOUR
            if grid[row][column].isOpen():
                colour = OPEN_COLOUR
            if grid[row][column].isClosed():
                colour = CLOSED_COLOUR
            if grid[row][column].isBest():
                colour = BEST_PATH_COLOUR
            pygame.draw.rect(
                display, colour,
                [(MARGIN + WIDTH) * column + MARGIN,
                 (MARGIN + HEIGHT) * row + MARGIN, WIDTH, HEIGHT])

# starting from end node follow parents to start node for best path
def drawShortestPath(grid, end):
    last = end
    for x in range(40):
        for y in range(40):
            if last.parent == start:
                break
            lastx, lasty = last.getMyPosition()
            cameFrom = grid[lastx][lasty].parent
            cfx, cfy = cameFrom.getMyPosition()
            if not grid[cfx][cfy].isStart():
                grid[cfx][cfy].makeBest()
            grid[cfx][cfx].draw(display)
            last = cameFrom

# reset the grid completely
def resetGrid(grid):
    for x in range(40):
        for y in range(40):
            grid[x][y].reset()
            grid[x][y].parent = None
            grid[x][y].hcost = 0
            grid[x][y].gcost = math.inf
            grid[x][y].fcost = 0

# reset the grid keeping the walls, start and end nodes
def resetForReRun(grid):
    for x in range(40):
        for y in range(40):
            if grid[x][y].isStart() or grid[x][y].isEnd() or grid[x][y].isWall():
                grid[x][y].parent = None
                grid[x][y].hcost = 0
                grid[x][y].gcost = math.inf
                grid[x][y].fcost = 0
            else:
                grid[x][y].reset()
                grid[x][y].parent = None
                grid[x][y].hcost = 0
                grid[x][y].gcost = math.inf
                grid[x][y].fcost = 0

# get the "manhattan distance" between two coordinates
# the absolute value of value 1 in the first coords 
# subtracted by the first value in the other 
# then the same for the second values
#then returning the sum of the resulting values
def getManhattanDistance(pt1, pt2):
    distance = 0
    for i in range(len(pt1)):
        distance += abs(pt1[i] - pt2[i])
    return distance

# starting from start node, evaluate neighboring nodes to 
# find best path to end node
#* algorithm is A*. This algorithm knows the 
#* location of the end node and makes its decision with another 
#* function that gets a heuristic for the distance between the 
#* current node and the end node. gives each node it discovers
#* a value of the distance from the start to the current node plus the heuristic
#* then, it chooses the current cheapest node, repeating untill end is found

def algorithm(grid, current, start, end):
    if not start:
        print("select a start node")
        return
    elif not end:
        print("select an end node")
        return
    start.gcost = 0
    start.fcost = getManhattanDistance(start.getMyPosition(), end.getMyPosition())
    heapq.heappush(open, start)
    if open:
        while open[0] != end:
            # get the best node from open set
            # and make that current node
            current = heapq.heappop(open)
            if not current.isStart() and not current.isEnd():
                current.makeClosed()
                current.draw(display)
            closed.append(current)
            neighbors = current.getNeighbors(grid)
            for neighbor in neighbors:
                cost = None
                cost = current.gcost + 10
                if neighbor in open and cost < neighbor.gcost:
                    neighbor.reset()
                    i = open.index(neighbor)
                    open[i] = open[-1]
                    open.pop()
                    heapq.heapify(open)
                if neighbor in closed and cost < neighbor.gcost:
                    closed.remove(neighbor)
                    neighbor.reset()
                if neighbor not in open and neighbor not in closed:
                    neighbor.gcost = cost        
                    neighbor.hcost = getManhattanDistance(neighbor.getMyPosition(), end.getMyPosition())
                    neighbor.fcost = neighbor.gcost + neighbor.hcost 
                    neighbor.parent = current
                    if not neighbor.isEnd():
                        neighbor.makeOpen()
                        neighbor.draw(display)
                    print("Neighbors for current. FCOST: ", neighbor.fcost)
                    heapq.heappush(open, neighbor) 
            if not open:
                    print("connot find end node")
                    return                
        drawShortestPath(grid, end)     
        print("FOUND YOU")  

# initialize the grid with Node objects
for x in range(40):
    grid.append([])
    for y in range(40):
        node = Node(x, y, WIDTH, HEIGHT)
        grid[x].append(node)

# init function for pygame
pygame.init()

# create display and set size. Currently using
#  883 x 883 as it fits 40 x 40 20 x 20 px squares
display = pygame.display.set_mode((883, 883))

# set caption of the display
pygame.display.set_caption('Pathfinding Visualizer - S for Start node - E for End - Left Click for wall - Right Click for reset selected - R reset all - SPACE to run')
clock = pygame.time.Clock()

# check for inputs and execute tasks based on said inputs
while not done:
    clock.tick(60)
    for event in pygame.event.get():
        # exit game 
        if event.type == pygame.QUIT:
            done = True
        # place walls
        elif pygame.mouse.get_pressed()[0]:
            placeWalls(start, end, grid)

        # reset selected node
        elif pygame.mouse.get_pressed()[2]:
            resetNode(start, end, grid)

        # keypress events
        elif event.type == pygame.KEYDOWN:
            # check if the start node has been placed if not, 
            # set node to star if it has been placed, remove 
            # the current node and place it in new location.
            if event.key == pygame.K_s:    
                row, col = getPosition()
                if not start:   
                    start = grid[row][col]
                    grid[row][col].makeStart()                 
                else:
                    for x in range(40):
                        for y in range(40):
                            if grid[x][y].isStart():
                                start = None
                                grid[x][y].reset()
                    if grid[row][col].isEnd():
                        end = None
                        start = grid[row][col]
                        grid[row][col].makeStart()
                    else:
                        start = grid[row][col]
                        grid[row][col].makeStart()
            elif event.key == pygame.K_e:
                row, col = getPosition()
                if not end:
                    end = grid[row][col]
                    grid[row][col].makeEnd()
                elif end:
                    for x in range(40):
                        for y in range(40):
                            if grid[x][y].isEnd():
                                end = None
                                grid[x][y].reset()
                            elif grid[row][col].isStart():
                                start = None
                                end = grid[row][col]
                                grid[row][col].makeEnd()
                            else:
                                end = grid[row][col]
                                grid[row][col].makeEnd()
            elif event.key == pygame.K_r:
                start = None
                end = None
                current = None
                closed = []
                open = []
                resetGrid(grid)
                rerun = False
            elif event.key == pygame.K_SPACE:
                if not rerun:
                    algorithm(grid, current, start, end)
                    rerun = True
                else:
                    current = None
                    closed = []
                    open = []
                    resetForReRun(grid)
                    draw()
                    pygame.display.update()
                    algorithm(grid, current, start, end)
    display.fill(MARGIN_COLOUR)
    # draw
    draw()
    pygame.display.update()

#quit fuction for pygame
pygame.quit()

select a start node
select a start node
select a start node
select a start node
Neighbors for current. FCOST:  62
Neighbors for current. FCOST:  62
Neighbors for current. FCOST:  64
Neighbors for current. FCOST:  64
Neighbors for current. FCOST:  71
Neighbors for current. FCOST:  71
Neighbors for current. FCOST:  73
Neighbors for current. FCOST:  71
Neighbors for current. FCOST:  73
Neighbors for current. FCOST:  75
Neighbors for current. FCOST:  75
Neighbors for current. FCOST:  75
Neighbors for current. FCOST:  80
Neighbors for current. FCOST:  80
Neighbors for current. FCOST:  82
Neighbors for current. FCOST:  80
Neighbors for current. FCOST:  80
Neighbors for current. FCOST:  82
Neighbors for current. FCOST:  84
Neighbors for current. FCOST:  84
Neighbors for current. FCOST:  86
Neighbors for current. FCOST:  86
Neighbors for current. FCOST:  86
Neighbors for current. FCOST:  86
Neighbors for current. FCOST:  89
Neighbors for current. FCOST:  89
Neighbors for current. FCOST:  89
Ne